# CellOracle GRN

# Set-up

In [1]:
import pandas as pd
import muon as mu
import scanpy as sc
import celloracle as co
import numpy as np
import matplotlib.pyplot as plt

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dn

In [114]:
# Load args
path_data = "/cellar/users/aklie/data/datasets/paul15/annotated/Paul_etal_15.h5mu"
path_r2g = None
path_tf2r = None
path_base_grn = "/cellar/users/aklie/opt/gene_program_evaluation/src/inference/grn_models/celloracle/resources/base_grns/mm9_mouse_atac_atlas_data_TSS_and_cicero_0.9_accum_threshold_10.5_DF_peaks_by_TFs_v202204.parquet"
path_additional_grn = "/cellar/users/aklie/data/datasets/paul15/ref/TF_data_in_Paul15.csv"
cluster_key = "louvain_annot"
layer = "imputed_count"
alpha = 10
bagging_number = 20
scale = True
path_out = "/cellar/users/aklie/data/datasets/paul15/analysis/celloracle/grn.csv"

# Prep base GRN

In [116]:
# Process base GRN
if path_base_grn:
    print("Loading base GRN")
    base_grn = pd.read_parquet(path_base_grn)
else:
    r2g = pd.read_csv(path_r2g)
    tfb = pd.read_csv(path_tf2r)
    if (r2g.shape[0] == 0) or (tfb.shape[0] == 0):
        grn = pd.DataFrame(columns=['source', 'target', 'score', 'pval'])
        grn.to_csv(path_out, index=False)
        exit()
    tfb['score'] = 1
    r2g = r2g[['cre', 'gene']]
    base_grn = pd.merge(
        r2g,
        tfb
        .pivot(index='cre', columns='tf')
        .fillna(0)
        .droplevel(0, axis=1)
        .reset_index()
    )
    base_grn = base_grn.rename(columns={'cre': 'peak_id', 'gene': 'gene_short_name'})
    base_grn['peak_id'] = base_grn['peak_id'].str.replace('-', '_')
    
# Make dictionary to add: dictionary key is TF and dictionary value is list of target genes.
if path_additional_grn:
    Paul_15_data = pd.read_csv(path_additional_grn)
    TF_to_TG_dictionary = {}

    for TF, TGs in zip(Paul_15_data.TF, Paul_15_data.Target_genes):
        # convert target gene to list
        TG_list = TGs.replace(" ", "").split(",")
        # store target gene list in a dictionary
        TF_to_TG_dictionary[TF] = TG_list

    # We invert the dictionary above using a utility function in celloracle.
    TG_to_TF_dictionary = co.utility.inverse_dictionary(TF_to_TG_dictionary)
else:
    TG_to_TF_dictionary = None

Loading base GRN


  0%|          | 0/178 [00:00<?, ?it/s]

# Set-up GEM

In [117]:
# Init object
mdata = mu.read(path_data)
adata = mdata.mod["rna"].copy()
adata.obs[cluster_key] = mdata.obs[cluster_key].copy()
if layer in adata.layers:
    print(f"Using data in layer {layer} for regression.")
    adata.X = adata.layers[layer].copy()
else:
    print(f"Could not find layer {layer}. Loading from CellOracle.")
    adata = co.data.load_Paul2015_data()

    # Instantiate Oracle object
    oracle = co.Oracle()

    # In this notebook, we use the unscaled mRNA count for the input of Oracle object.
    adata.X = adata.layers["raw_count"].copy()

    # Instantiate Oracle object.
    oracle.import_anndata_as_raw_count(adata=adata,
                                    cluster_column_name="louvain_annot",
                                    embedding_name="X_draw_graph_fa")

    # Add base GRN
    oracle.import_TF_data(TF_info_matrix=base_grn)

    # Perform PCA
    oracle.perform_PCA()

    # Select important PCs
    n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
    n_comps = min(n_comps, 50)

    # Perform knn imputation
    n_cell = oracle.adata.shape[0]
    k = int(0.025*n_cell)
    oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8, b_maxl=k*4, n_jobs=4)

    # Grab adata
    adata = oracle.adata
    adata.X = adata.layers["imputed_count"].copy()

Using data in layer imputed_count for regression.


# Build GRN manually

In [118]:
# Model TF ~ G for every cluster
cluster_grns = {}
for cluster in adata.obs[cluster_key].cat.categories:
    print(f"Building GRN for {cluster}")
    adata_sub = adata[adata.obs[cluster_key] == cluster].copy()
    net = co.Net(
        gene_expression_matrix=adata_sub.to_df(), # Input gene expression matrix as data frame
        TFinfo_matrix=base_grn, # Input base GRN
        verbose=True
    )
    if TG_to_TF_dictionary:
        print("Adding additional TF-TG pairs to the base GRN")
        net.addTFinfo_dictionary(TF_to_TG_dictionary)
    net.fit_All_genes(
        bagging_number=bagging_number,
        alpha=alpha,
        scaling=scale,
        verbose=True
    )
    net.updateLinkList(verbose=True)
    inference_result = net.linkList.copy()
    cluster_grns[cluster] = inference_result
    print(f"Finished building GRN for {cluster}")

Building GRN for Ery_0
initiating Net object ...
gem_shape: (73, 1999)
TF info shape: (91976, 1095)
initiation completed.
Adding additional TF-TG pairs to the base GRN


  0%|          | 0/1849 [00:00<?, ?it/s]

  0%|          | 0/1847 [00:00<?, ?it/s]

Finished building GRN for Ery_0
Building GRN for Ery_1
initiating Net object ...
gem_shape: (138, 1999)
TF info shape: (91976, 1095)
initiation completed.
Adding additional TF-TG pairs to the base GRN


  0%|          | 0/1849 [00:00<?, ?it/s]

...

  0%|          | 0/1847 [00:00<?, ?it/s]

Finished building GRN for Mo_1
Building GRN for Mo_2
initiating Net object ...
gem_shape: (202, 1999)
TF info shape: (91976, 1095)
initiation completed.
Adding additional TF-TG pairs to the base GRN


  0%|          | 0/1849 [00:00<?, ?it/s]

  0%|          | 0/1847 [00:00<?, ?it/s]

Finished building GRN for Mo_2


In [119]:
# Extract grn
grn = pd.concat([v.assign(cluster=k) for k, v in cluster_grns.items()])
grn = grn.dropna()[['source', 'target', 'coef_mean', 'p', 'cluster']]
grn = grn.rename(columns={'coef_mean': 'score', 'p': 'pval'})
grn = grn.rename(columns={'source': 'tf', 'target': 'gene'})
grn = grn.sort_values(['tf', 'score'], ascending=[True, False])

In [120]:
# Write
grn.to_csv(path_out, index=False)

In [122]:
mdata.uns["grn"] = grn

In [123]:
mdata.write("/cellar/users/aklie/data/datasets/paul15/analysis/celloracle/2024_05_21/celloracle.h5mu")

# Extract from CellOracle object

In [199]:
links = co.data.load_tutorial_links_object()
links

In [205]:
test = pd.concat([v.assign(cluster=k) for k, v in links.links_dict.items()])

In [206]:
test[test["cluster"] == "Mo_2"]

,source,target,coef_mean,coef_abs,p,-logp,cluster
0,Irf7,0610007L01Rik,0.007751,0.007751,8.629981e-05,4.063990,Mo_2
1,Fli1,0610007L01Rik,0.022571,0.022571,1.173783e-09,8.930412,Mo_2
2,Foxp1,0610007L01Rik,0.011244,0.011244,3.803174e-05,4.419854,Mo_2
3,Chd2,0610007L01Rik,-0.013774,0.013774,6.197509e-10,9.207783,Mo_2
4,Stat1,0610007L01Rik,0.015618,0.015618,4.605829e-06,5.336692,Mo_2
...,...,...,...,...,...,...,...
74932,Stat3,Zyx,0.027406,0.027406,5.937966e-14,13.226362,Mo_2
74933,Nfic,Zyx,0.001807,0.001807,2.059337e-01,0.686272,Mo_2
74934,Stat5a,Zyx,0.018760,0.018760,1.559392e-10,9.807045,Mo_2
74935,Hnf4a,Zyx,-0.008124,0.008124,1.052843e-05,4.977636,Mo_2


In [303]:
# Extract tutorial_grn
tutorial_grn = pd.concat([v.assign(cluster=k) for k, v in links.links_dict.items()])
tutorial_grn = tutorial_grn.dropna()[['source', 'target', 'coef_mean', 'p', 'cluster']]
tutorial_grn = tutorial_grn.rename(columns={'coef_mean': 'score', 'p': 'pval'})
tutorial_grn = tutorial_grn.rename(columns={'source': 'tf', 'target': 'gene'})
tutorial_grn

,tf,gene,score,pval,cluster
0,Irf7,0610007L01Rik,-0.009400,4.912392e-08,Ery_0
1,Fli1,0610007L01Rik,0.000347,8.124897e-01,Ery_0
2,Foxp1,0610007L01Rik,-0.007035,3.098882e-04,Ery_0
3,Chd2,0610007L01Rik,-0.010569,5.736120e-06,Ery_0
4,Stat1,0610007L01Rik,0.000632,3.694124e-01,Ery_0
...,...,...,...,...,...
74932,Stat3,Zyx,0.027406,5.937966e-14,Mo_2
74933,Nfic,Zyx,0.001807,2.059337e-01,Mo_2
74934,Stat5a,Zyx,0.018760,1.559392e-10,Mo_2
74935,Hnf4a,Zyx,-0.008124,1.052843e-05,Mo_2


In [304]:
tutorial_grn[tutorial_grn["cluster"]=="Mo_2"]

,tf,gene,score,pval,cluster
0,Irf7,0610007L01Rik,0.007751,8.629981e-05,Mo_2
1,Fli1,0610007L01Rik,0.022571,1.173783e-09,Mo_2
2,Foxp1,0610007L01Rik,0.011244,3.803174e-05,Mo_2
3,Chd2,0610007L01Rik,-0.013774,6.197509e-10,Mo_2
4,Stat1,0610007L01Rik,0.015618,4.605829e-06,Mo_2
...,...,...,...,...,...
74932,Stat3,Zyx,0.027406,5.937966e-14,Mo_2
74933,Nfic,Zyx,0.001807,2.059337e-01,Mo_2
74934,Stat5a,Zyx,0.018760,1.559392e-10,Mo_2
74935,Hnf4a,Zyx,-0.008124,1.052843e-05,Mo_2


In [305]:
mdata.uns["tutorial_grn"] = tutorial_grn

In [306]:
mdata.write("/cellar/users/aklie/data/datasets/paul15/analysis/celloracle/2024_05_21/celloracle.h5mu")

# DONE!

---